# Quantum Hamiltonians as matrix product operators

In this tutorial, we will first introduce so-called operator chains, and demonstrate how to convert them to MPO form. This mechanism facilitates a concise construction of quantum Hamiltonians as MPOs.

In [1]:
# import NumPy and the PyTeNet package
import numpy as np
import pytenet as ptn

## Operator chains

An "operator chain" defined in `opchain.py` consists of local operators forming an outer product, i.e., $op_i \otimes op_{i+1} \otimes \cdots \otimes op_{i+n-1}$, with $op_i$ acting on lattice site $i$ (and identity operations on the remaining lattice sites). Here is a basic illustration:

In [2]:
# Pauli matrices
sigma_x = np.array([[0.,  1.], [1.,  0.]])
sigma_y = np.array([[0., -1j], [1j,  0.]])
sigma_z = np.array([[1.,  0.], [0., -1.]])

# create the operator chain id x id x sigmaX x sigmaX x sigmaZ x ...
opchain = ptn.OpChain(oplist=[sigma_x, sigma_x, sigma_z], istart=2, qD=[0, 0])
# (the qD parameter contains quantum numbers to be sandwiched between the operators,
#  see the section below for an explanation)

Our `opchain` simply stores the arguments used for constructing it as instance variables:

In [3]:
opchain.oplist

[array([[0., 1.],
        [1., 0.]]),
 array([[0., 1.],
        [1., 0.]]),
 array([[ 1.,  0.],
        [ 0., -1.]])]

In [4]:
opchain.istart

2

In [5]:
opchain.qD

[0, 0]

The `as_matrix()` function forms the outer product, i.e., the matrix representation of the operator chain:

In [6]:
# local dimension
d = 2
# number of lattice sites
L = 8
A = opchain.as_matrix(d, L)
type(A)

numpy.ndarray

In [7]:
# the matrix dimension is d^L x d^L
A.shape

(256, 256)

In [8]:
# here's what the as_matrix() function actually computes:
id2 = np.identity(2)
A1 = np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(
        id2, id2), sigma_x), sigma_x), sigma_z), id2), id2), id2)
# compare
np.linalg.norm(A1 - A)

0.0

## Convert operator chains to MPOs

How can we construct a MPO version of a sum of operator chains? A useful mental picture are railway tracks running from east to west (i.e., from the rightmost to the leftmost lattice site), possibly interconnected by switches. Each operator chain corresponds to a train running once from east to west along one route through the tracks. (There is no risk of collision since trains operate at different hours.) Markers are placed at uniform intervals besides the tracks, each displaying a local operator (like the Pauli matrices in the example above). The operator chain (train) collects the markers which it encounters during its ride (preserving order).

We can exploit the locality of typical operator chains (e.g., acting non-trivially only on two neighboring lattice sites) by creating a special "identity" track: its markers solely display the local identity operation. Operator trains share this identity track before and after the sites which they act on.

Translating this description to MPOs, the number of tracks running in parallel (at a given longitude) is precisely the virtual bond dimension.

The `from_opchains()` classmethod of the `MPO` class implements this algorithm. (The main part is only around 40 lines of Python code in case you want to have a look.) Let's demonstrate it by a short example:

In [9]:
# first create two additional operator chains:
# id x sigmaY x sigmaZ x ...
opchain2 = ptn.OpChain(oplist=[sigma_y, sigma_z], istart=1, qD=[0])
# id x id x id x id x sigmaX x ...
opchain3 = ptn.OpChain(oplist=[sigma_x], istart=4, qD=[])

In [10]:
# physical quantum numbers (see below)
qd = d * [0]

# construct MPO representation of the sum of the operator chains
mpo = ptn.MPO.from_opchains(qd, L, [opchain, opchain2, opchain3])

In [11]:
# consistency check
np.linalg.norm(mpo.as_matrix() -
               (opchain.as_matrix(d, L) + opchain2.as_matrix(d, L) + opchain3.as_matrix(d, L)))

0.0

In [12]:
# need at most 3 virtual bonds
mpo.bond_dims

[1, 1, 2, 3, 3, 1, 1, 1, 1]

## Construct quantum Hamiltonians

The module `hamiltonian.py` uses the mechanism to construct quantum Hamiltonians as MPOs. As illustration, for the Ising Hamiltonian

$$
H = \sum_{j=1}^{L-1} J\,\sigma^z_j \sigma^z_{j+1} + \sum_{j=1}^L \left( h\,\sigma^z_j + g\,\sigma^x_j \right)
$$

(where we have omitted the $\otimes$ symbol between $\sigma^z_j$ and $\sigma^z_{j+1}$ for brevity), the function `ising_MPO(L, J, h, g)` in `hamiltonian.py` creates an operator chain for $J\,\sigma^z_0 \sigma^z_1$ and one for $h\,\sigma^z_0 + g\,\sigma^x_0$, and then calls `local_opchains_to_MPO()` to shift them along the lattice (i.e., perform the sum over $j$). The same procedure works for other quantum Hamiltonians, and is not restricted to nearest neighbor interactions.

In [13]:
# Hamiltonian parameters
J =  1.0
h = -0.4
g =  0.7
# construct Ising Hamiltonian as MPO
mpo_ising = ptn.ising_mpo(L, J, h, g)

In [14]:
# virtual bond dimensions
mpo_ising.bond_dims

[1, 3, 3, 3, 3, 3, 3, 3, 1]

## Quantum numbers and conservation laws

Let's discuss how (additive) quantum numbers enter the story so far. In many cases, a Hamiltonian respects conservation laws, like preserving the total spin or particle number of a physical system, for example. How to exploit this within the matrix product operator formalism is not entirely obvious, but here is how it works: we first identify the physical quantum numbers at each lattice site, like $\pm \frac{1}{2}$ for spin-up or spin-down. (PyTeNet stores them in variables denoted `qd`.) Likewise, each virtual bond is also associated with a quantum number (`qD` in PyTeNet, see below). Now every MPO tensor obeys a sparsity pattern dictated by the quantum numbers: The sum of first physical and left virtual bond quantum number of each non-zero tensor entry is equal to the sum of second physical and right virtual bond quantum number. (Here "first" and "second" refers to the row and column dimension of a site-local operator.) PyTeNet provides the `is_qsparse` utility function to probe such a sparsity pattern. Note that we do not have to manually enforce it; instead, the sparsity pattern appears naturally when constructing the MPO representation of a Hamiltonian, and quantum numbers provide the means for actually recognizing and understanding it. In practical terms, quantum numbers allow to optimize common operations like singular value decompositions via partitioning into non-zero blocks.

How can we obtain the virtual bond quantum numbers in the first place? Let's illustrate this via the XXZ Heisenberg model, with Hamiltonian represented as

$$
H = \sum_{j=1}^{L-1} \left( \tfrac{1}{2} J\,S^{+}_j S^{-}_{j+1} + \tfrac{1}{2} J\,S^{-}_j S^{+}_{j+1} + \Delta\,S^z_j S^z_{j+1} \right) - \sum_{j=1}^L h\,S^z_j
$$

where $S^{\pm}_j$ are the spin raising and lowering operators at site $j$, respectively, and $S^z_j = \frac{1}{2} \sigma^z_j$. As one might guess, the raising and lowering operators change the spin quantum number by $1$ (such that the overall net effect of both $S^{+}_j S^{-}_{j+1}$ and $S^{-}_j S^{+}_{j+1}$ is zero). We can translate this into code by sandwiching the virtual bond quantum number $1$ between $S^{+}_j$ and $S^{-}_{j+1}$, and likewise $-1$ between $S^{-}_j$ and $S^{+}_{j+1}$. (To avoid any numerical rounding issues, quantum numbers are generally stored as integers, and thus we multiply all spin quantum numbers by $2$ in the code.) Concretely, the corresponding operator chains read `OpChain(oplist=[0.5*J*Sup, Sdn], qD=[2])` and `OpChain(oplist=[0.5*J*Sdn, Sup], qD=[-2])`, respectively. The `from_opchains()` classmethod described above automatically copies the virtual bond quantum numbers from the operator chains, and voilà, the MPO representation of the Hamiltonian is endowed with quantum numbers.

In [15]:
# Hamiltonian parameters
J =  1.0
D =  0.8
h = -0.1
# construct XXZ Heisenberg Hamiltonian as MPO
mpo_xxz = ptn.heisenberg_xxz_mpo(L, J, D, h)

In [16]:
# physical quantum numbers (multiplied by 2)
mpo_xxz.qd

array([ 1, -1])

In [17]:
# virtual bond dimensions
mpo_xxz.bond_dims

[1, 5, 5, 5, 5, 5, 5, 5, 1]

In [18]:
# virtual bond quantum numbers of third bond (multiplied by 2)
mpo_xxz.qD[3]

array([ 0,  2, -2,  0,  0])

In [19]:
# sparsity pattern consistency check
ptn.is_qsparse(mpo_xxz.A[3], [mpo_xxz.qd, -mpo_xxz.qd, mpo_xxz.qD[3], -mpo_xxz.qD[4]])

True

Note that we can effectively disable quantum numbers by setting them all to zero, since the above sparsity pattern rule is then always trivially satisfied.